### About this result

Analyzing the results runnning on IBM quantum platform, the result data is stored in `./pred_outputs/ibmq` with `npy` format.

In [ ]:
import os
from itertools import product
import json

import numpy as np

with open("config.json", "r") as json_file:
    json_config = json.load(json_file)
    ibmq_dir = os.path.join(json_config["predictions_dir"], "ibmq")

In [ ]:
shots = 1024
num_pt_ptcs = 2
num_bin_data = 100
backend_list = ["ibmq_mumbai"]
channel_list = ["BB-QCD", "TT-QCD"]
rnd_seed_list = range(3)

def read_ibmq_npy(backend, abbrev, num_nr_qubits, rnd_seed):
    """Read the IBMQ result (npy file)."""

    global shots, num_pt_ptcs, num_bin_data

    for ibmq_file in os.listdir(ibmq_dir):
        check_backend = f"{backend}-s{shots}" in ibmq_file
        check_data = f"{abbrev}_r{rnd_seed}-p{num_pt_ptcs}_n{num_bin_data}" in ibmq_file
        check_model = f"{num_nr_qubits}_1_{num_nr_qubits}" in ibmq_file
        if check_backend and check_data and check_model:
            npy_path = os.path.join(ibmq_dir, ibmq_file)
            break
    else:
        raise FileNotFoundError(
            "Not found file with ->\n"
            f" * {backend}-s{shots}\n"
            f" * {abbrev}_r{rnd_seed}-p{num_pt_ptcs}_n{num_bin_data}\n"
            f" * {num_nr_qubits}_1_{num_nr_qubits}\n"
        )
    result = np.load(npy_path, allow_pickle=True)
    result = result.item()
    return result

def read_noise_npy(abbrev, num_nr_qubits, rnd_seed):
    """Read the IBMQ result (npy file)."""

    global num_pt_ptcs, num_bin_data

    for ibmq_file in os.listdir(ibmq_dir):
        check_noise = f"noise" in ibmq_file
        check_data = f"{abbrev}_r{rnd_seed}-p{num_pt_ptcs}_n{num_bin_data}" in ibmq_file
        check_model = f"{num_nr_qubits}_1_{num_nr_qubits}" in ibmq_file
        if check_noise and check_data and check_model:
            npy_path = os.path.join(ibmq_dir, ibmq_file)
            break
    else:
        raise FileNotFoundError(
            "Not found file with ->\n"
            f" * noise-\n"
            f" * {abbrev}_r{rnd_seed}-p{num_pt_ptcs}_n{num_bin_data}\n"
            f" * {num_nr_qubits}_1_{num_nr_qubits}\n"
        )
    result = np.load(npy_path, allow_pickle=True)
    result = result.item()
    return result

In [ ]:
noise_prod_list = [1E-1, 5E-2, 1E-2, 5E-3, 1E-3]

for abbrev, backend, rnd_seed in product(channel_list, backend_list, rnd_seed_list):
    print(f"* Channel: {abbrev} | Backend: {backend} | Size: {100} | Random seed: {rnd_seed}\n")
    print(f"          | SIM  ACC | IBMQ ACC | S/I  ACC | S/I DIFF |", end="")
    print("|".join([f"  {noise_prod:.0e}   " for noise_prod in noise_prod_list]))

    for num_nr_qubits in [3, 5, 7]:
        # Horizontal splitting line.
        print("-" * 120)
        
        # Read npy file.
        result = read_ibmq_npy(backend, abbrev, num_nr_qubits, rnd_seed)
        result.update(read_noise_npy(abbrev, num_nr_qubits, rnd_seed))

        # Prepare true labels and predictions.
        y_true = result["y_true"].reshape(-1).numpy()
        y_penl = result["y_penl"].reshape(-1).detach().numpy()
        y_ibmq = result["y_ibmq"].reshape(-1).detach().numpy()

        # Noise accuracy
        y_noise = [
            result[f"y_noise_{noise_prod:.0e}"].reshape(-1).detach().numpy() \
            for noise_prod in noise_prod_list
        ]

        # Calculate accuracy and loss (difference).
        acc_list = []
        acc_list.append((y_true == (y_penl > 0)).mean()) # Simulation accuracy.
        acc_list.append((y_true == (y_ibmq > 0)).mean()) # IBMQ accuracy.
        acc_list.append(((y_penl > 0) == (y_ibmq > 0)).mean()) # S/I accuracy.
        acc_list.append(np.mean(np.abs(y_penl - y_ibmq))) # S/I diff.

        # Add noise accuracy.
        acc_list += [(y_true == (_y_noise > 0)).mean() for _y_noise in y_noise]

        print(f"  Q = {num_nr_qubits}   |", end="")
        print("|".join([f"   {acc:.2f}   " for acc in acc_list]))
    
    print("\n" + "#" * 120 + "\n")